In [96]:
import pandas as pd
import matplotlib.pyplot as plt
import re
from scipy.sparse import csr_matrix, coo_matrix
import numpy as np

%matplotlib inline

In [97]:
ratings = pd.read_csv('BX-Book-Ratings.csv', encoding='iso-8859-1', sep = ';')
ratings.columns = ['user_id', 'isbn', 'book_rating']
books = pd.read_csv('BX-Books.csv', sep=';', encoding = 'iso-8859-1', dtype =str)

books["Book-Title"].nunique() == books["ISBN"].nunique()
book_dict = books[["Book-Title","ISBN"]].set_index("Book-Title").to_dict()["ISBN"]
books['new_isbn'] = books["Book-Title"].apply(lambda x: book_dict[x])
books["Book-Title"].nunique() == books["new_isbn"].nunique()
books['isbn'] = books['new_isbn']

del books['Image-URL-L']
del books['Image-URL-M']
del books['Image-URL-S']
del books['Book-Author']
del books['Publisher']
del books['ISBN']
del books['new_isbn']

newdf = ratings[ratings.book_rating>0]
joined = books.merge(newdf, on ='isbn')
print(newdf.shape)

(433671, 3)


In [104]:
bookinfo = pd.read_csv("goodreads_list_props.csv")
bookinfo.drop_duplicates(inplace = True)

In [105]:
books.drop_duplicates(subset = 'isbn',inplace = True)

In [108]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import sys


def is_isbn10_valid(isbn):
    """
    Check ISBN-10 is valid.
    Code Implementaion from:
    http://en.wikipedia.org/wiki/International_Standard_Book_Number
    """
    if len(isbn) != 10:
        return False
    if ((not isbn[0:9].isdigit()) or
            ((isbn[-1] != 'X') and (not isbn[-1].isdigit()))):
        return False
    result = sum((10 - i) * (int(x) if x != 'X' else 10)
                 for i, x in enumerate(isbn))
    return result % 11 == 0


def is_isbn13_valid(isbn):
    """
    Check ISBN-13 is valid.
    Code Implemetation from:
    http://en.wikipedia.org/wiki/International_Standard_Book_Number
    """
    if len(isbn) != 13 or isbn.isdigit() is not True:
        return False
    check = (10 - (sum(int(digit) * (3 if idx % 2 else 1)
                       for idx, digit in enumerate(isbn[:12])) % 10)) % 10
    return check == int(isbn[-1])


def isbn13_to_isbn10(isbn13_str):
    """
    Convert ISBN-13 to ISBN-10.
    """
    num = 11 - sum((10 - i) * (int(x))
                   for i, x in enumerate(isbn13_str[3:12])) % 11
    if num == 10:
        check_digit = 'X'
    elif num == 11:
        check_digit = 0
    else:
        check_digit = num
    return isbn13_str[3:12] + str(check_digit)


def isbn10_to_isbn13(isbn10_str):
    """
    Convert ISBN-10 to ISBN-13.
    """
    check_digit = (
        10 - (sum(int(digit) * (3 if idx % 2 else 1)
                  for idx, digit in enumerate('978' + isbn10_str[:9])
                  ) % 10)) % 10
    return '978' + isbn10_str[:9] + str(check_digit)


def isbn_converter(isbn):
    """
    Convert isbn format to another format.
    """
    if is_isbn10_valid(isbn):
        result = isbn10_to_isbn13(isbn)
    elif is_isbn13_valid(isbn):
        result = isbn13_to_isbn10(isbn)
    else:
        return None
    return result


if __name__ == "__main__":
    for isbn_str in sys.argv[1:]:
        the_result = isbn_converter(isbn_str)
        if the_result:
            print(the_result)
        else:
            print("Bad ISBN " + isbn_str)


Bad ISBN -f
Bad ISBN C:\Users\vijay\AppData\Roaming\jupyter\runtime\kernel-bf3aea00-a23d-4c8e-a135-49b95a56c649.json


In [109]:
isbn13 = []
for i in books['isbn']:
    isbn13.append(isbn_converter(i))

In [111]:
books['isbn13'] = isbn13

In [117]:
books.dropna(subset = ['isbn13'],inplace = True)
bookinfo.dropna(subset = ['isbn13'],inplace = True)

In [120]:
mergedinfo = bookinfo.merge(books,on = 'isbn13',how = 'inner')

In [122]:
mergedinfo

,book_name,author,rating,votes,description,book_type,no_of_pages,first_published,isbn13,genre,link,Book-Title,Year-Of-Publication,isbn
0,The Kite Runner,Khaled Hosseini,4.25,1791758,"<i> “It may be unfair, but what happens in a f...",Paperback,391.0,['May 29th 2003'],9781594480003,Fiction,https://www.goodreads.com//book/show/77203.The...,The Kite Runner,2004,1594480001
1,The Old Man and the Sea,Ernest Hemingway,3.72,534937,It is the story of an old Cuban fisherman and ...,Hardcover,132.0,['1952'],9780684830490,Classics,https://www.goodreads.com//book/show/2165.The_...,Old Man And The Sea (Scribner Classics),1996,0684830493
2,The Pillars of the Earth,Ken Follett,4.29,474471,<i>For the Movie tie-in edition with the same ...,Paperback,973.0,['October 1st 1989'],9780451207142,Historical Fiction,https://www.goodreads.com//book/show/5043.The_...,Pillars of the Earth,1989,0451207149
3,Les Misérables,Victor Hugo,4.14,552584,Introducing one of the most famous characters ...,Mass Market Paperback,1463.0,['1862'],9780451525260,Classics,https://www.goodreads.com//book/show/24280.Les...,Les Miserables a New Unabridged Translation (S...,1987,0451525264
4,Don Quixote,Miguel de Cervantes Saavedra,3.85,153081,Don Quixote has become so entranced by reading...,Paperback,1023.0,['January 16th 1605'],9780142437230,Classics,https://www.goodreads.com//book/show/3836.Don_...,Don Quixote: The Ingenious Hidalgo De LA Manch...,2003,0142437239
5,The Giving Tree,Shel Silverstein,4.38,676945,"""Once there was a tree...and she loved a littl...",Hardcover,64.0,['1964'],9780060256654,Childrens,https://www.goodreads.com//book/show/370493.Th...,The Giving Tree,1964,0060256656
6,The Stand,Stephen King,4.34,448747,This is the way the world ends: with a nanosec...,Hardcover,1153.0,['1978'],9780385199575,Horror,https://www.goodreads.com//book/show/149267.Th...,The Stand (The Complete and Uncut Edition),1990,0385199570
7,"Moby-Dick; or, The Whale",Herman Melville,3.45,384478,"“Where the White Whale, yo?”<br/><br/>Ah, my f...",Paperback,720.0,['October 18th 1851'],9780142437247,Classics,https://www.goodreads.com//book/show/153747.Mo...,"Moby-Dick: Or, the Whale (Penguin Classics)",2003,0142437247
8,A Clockwork Orange,Anthony Burgess,3.97,435629,"A vicious fifteen-year-old ""droog"" is the cent...",Paperback,192.0,['December 1962'],9780393312836,Science Fiction,https://www.goodreads.com//book/show/227463.A_...,A Clockwork Orange (Norton Paperback Fiction),1986,0393312836
9,The Brothers Karamazov,Fyodor Dostoyevsky,4.30,170528,"If there was still any doubt, let me confirm t...",Paperback,795.0,['November 1880'],9780374528379,Classics,https://www.goodreads.com//book/show/4934.The_...,The Brothers Karamazov,1984,0374528373
